#  California road network


In this project, I worked on [California Road Network](https://www.cs.utah.edu/~lifeifei/SpatialDataset.htm) dataset as practice for reading and cleaning big datasets. I created csv files for further analysis, but in this part I only read the datasets and created csv files to make dataset ready for analysis. 

Datasets include 
1. California Road Network's Nodes (Node ID, Longitude, Latitude)
2. California Road Network's Edges (Edge ID, Start Node ID, End Node ID, L2 Distance)
3. California's Points of Interest With Original Category Name (Category Name, Longitude, Latitude)
4. California's Points of Interest (Longitude, Latitude, Category ID)
5. Merge Points of Interest with Road Network--Map Format
For the 5th dataset, map format in detail is:
        For each edge:
        Start Node ID, End Node ID, Number of Points on This Edge, Edge Length. 
            For each point on this edge: 
            Category ID, Distance of This Point to the Start Node of This Edge  

For the first four datasets, I used ``pd.read_csv()`` command from pandas, but for the last file, I created generators for reading data. With this my aim was to clean data while reading and also keep memory usage in minimum. I created 3 csv files from the 5th file. 
    First file called "nodes_POIs.csv" was for spliting beginning node and related POIs in each line, but this dataset was also needed to be cleaned. So I created second csv as cleaned version of the first one which called "nodes_POIs2.csv". This dataset splits all POIs into different lines with keeping their beggining node as first in line. And as last csv file called "edge_lengths.csv" I only took into account beggining and end nodes of edges and the legth of them and also number of POIs on these edges while creating csv rows. 

---

> Burcin Sarac <br />


I started with importing required packages and setting working directory of mine. 

In [3]:
import pandas as pd
import itertools
import re
import os

In [4]:
os.chdir("E:/dersler/3.Spring/mining big datasets/Assignment_2")

## 1.Dataset

Saved as "nodes.csv" to the working directory. 

In [5]:
nodes = pd.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/cal.cnode", sep=' ', 
                    names= ["node_ID", "longitude", "latitude"])

In [6]:
nodes.head()

,node_ID,longitude,latitude
0,0,-121.904167,41.974556
1,1,-121.902153,41.974766
2,2,-121.896790,41.988075
3,3,-121.889603,41.998032
4,4,-121.886681,42.008739


In [7]:
nodes.to_csv("nodes.csv", index = False)

## 2.Dataset

Saved as "edges.csv" to the working directory. 

In [8]:
edges = pd.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/cal.cedge", sep=' ', 
                    names= ["edge_ID", "startnodeID", "endnodeID", "distance"])

In [9]:
edges.head()

,edge_ID,startnodeID,endnodeID,distance
0,0,0,1,0.002025
1,1,0,6,0.005952
2,2,1,2,0.014350
3,3,2,3,0.012279
4,4,3,4,0.011099


In [10]:
edges.to_csv("edges.csv", index = False)

## 3. Dataset


In [11]:
catnames = pd.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/CA", sep=' ', 
                    names= ["categoryname", "longitude", "latitude"])

In [12]:
catnames.head()

,categoryname,longitude,latitude
0,airport,-114.18639,34.30806
1,airport,-114.43083,34.52750
2,airport,-114.52667,33.86944
3,airport,-114.57528,34.18389
4,airport,-114.60194,34.81944


## 4. Dataset

In [14]:
pois = pd.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/caldata", sep=' ', 
                    names= ["longitude", "latitude", "catID"])

In [15]:
pois.head()

,longitude,latitude,catID
0,-114.186394,34.308060,0
1,-114.430832,34.527500,0
2,-114.526672,33.869438,0
3,-114.575279,34.183891,0
4,-114.601936,34.819439,0


## Merging 3th and 4th dataset as one

In this step I merged category id numbers with category names by using similarity between latitude and longitude values, but since the values has different digit numbers, I rounded them first. Afterwards, I filtered duplications and saved it as "categories.csv" to the working directory. 

In [33]:
merged_cat = pd.merge(pois, catnames,  how='outer', left_on=[round(pois.iloc[:,0],5),round(pois.iloc[:,1],5)], 
                      right_on = [round(catnames.iloc[:,1],5),
                                  round(catnames.iloc[:,2],5)]).drop(["longitude_x","latitude_x",
                                                                          "longitude_y","latitude_y"], 
                                                                     axis=1).drop_duplicates("key_0")

In [40]:
categories = merged_cat.drop(["key_0", "key_1"], axis=1).drop_duplicates("catID").reset_index(drop=True).dropna()

In [41]:
categories.head()

,catID,categoryname
0,0.0,airport
1,1.0,arch
2,2.0,area
3,3.0,arroyo
4,4.0,bar


In [42]:
categories.to_csv("categories.csv", index = False)

## 5. Dataset

For this dataset, as I mentioned in the beginning part, I created generators to read lines one by one instead of importing all dataset and load to memory. But first I downloaded the link and save it as original txt file with name "calmap.txt". 
Saved as "catnames.csv" to the working directory. 

In [17]:
import requests

outF = open("calmap.txt","w")
outF.writelines(requests.get("https://www.cs.utah.edu/~lifeifei/research/tpq/calmap.txt").text)

outF.close()

Afterwards, I created first generator below, which read the txt file by 2 lines in each step and while reading first line it merges numbers by using ``join()`` function. After that, it checks with the helps of if condition and filter the line if 4th number in each line is above 1(>1) in "calmap.txt" file, and then it merges beginning node, POIid and POI distance together by putting them into a line if condition was met.  With this process I created "nodes_POIs.csv" file, which includes beginning node ID and all POIs with distance values to the beginning node line by line. 

In second step, I splited all POIs into separate rows with their beginning node id numbers. 

For instance;

    In first step first line created like this;
        1,25,0.00537134,55,0.0040002,
        
    And in second step I converted it into this;
        1,25,0.00537134
        1,55,0.0040002
        
And at last step I just determined column names and save it again to the working directory. 
 

In [19]:
outF = open("nodes_POIs.csv", "w")

with open("calmap.txt","r") as f:
    for line1,line2 in itertools.zip_longest(*[f]*2):
        line1= ''.join(line1)
        if re.split(r"\s", str(line1))[3]!='':
            if float(re.split(r"\s", str(line1))[3])>=1:
                nodes_pois = ('{0},{1}'.format(re.split(r"\s", str(line1))[0], ','.join(re.split(r"\s", str(line2)))))
                outF.writelines(nodes_pois + '\n')

outF.close()

In [20]:
outF = open("nodes_POIs2.csv", "w")

with open("nodes_POIs.csv","r") as f:
    for line in f:
        for i in range(0,(len(re.split(",", str(line)))-2),2):
            if re.split(r",",str(line))[i+1] != '':
                nodes_pois = ('{0},{1},{2}'.format(re.split(",", str(line))[0],re.split(",", str(line))[i+1], 
                                           re.split(r",", str(line))[i+2]))
                outF.writelines(nodes_pois + '\n')

outF.close()

In [21]:
nodespois = pd.read_csv("nodes_POIs2.csv", names= ["startnodeID", "catID", "cat_distance"])
nodespois.to_csv("nodes_POIs2.csv", index = False)

After all I created edge_lenths data by only reading lines if 4th number in each line in "calmap.txt" dataset got values between 0 and 1. And append all lines which satisfied this condition with including their Start node, end node, edge lenth and number of POIs on that edge. 

In [22]:

outF = open("edge_lengths.csv", "w")

with open("calmap.txt","r") as f:
    for line1 in itertools.zip_longest(f):
        line1= ''.join(line1)
        if re.split(r"\s", str(line1))[3]!='':
            if not 0<float(re.split(r"\s", str(line1))[3])<1:
                edgelengths = (','.join(re.split(r"\s", str(line1))))
                outF.writelines(edgelengths+'\n')
                
outF.close()

In [32]:
edgedetails = pd.read_csv("edge_lengths.csv", names= ["startnode","endnode", "distance", "nr_of_pois"])
edgedetails.to_csv("edge_lengths.csv", index = False)

And after every documents had read, I used the command below to drop memory usage of Jupyter Notebook by decreasing its display limit. 

In [23]:
pd.set_option('display.max_columns', 50)